In [1]:
import pandas as pd
import numpy as np
import re

In [26]:
train_file = 'ALL_AML_grow.train.orig.txt'
test_file = 'ALL_AML_grow.test.orig.txt'
sample_file = 'table_ALL_AML_samples.txt'
train_idclass_file = 'ALL_AML_idclass.train.txt'
test_idclass_file = 'ALL_AML_idclass.test.txt'
train_tmp_file = 'LL_AML_grow.train.noaffy.tmp'
test_tmp_file = 'ALL_AML_grow.test.noaffy.tmp'
norm_train_file = 'ALL_AML_grow.train.norm.tmp'
norm_test_file = 'ALL_AML_grow.test.norm.tmp'
gcol_train_file = 'ALL_AML_gcol.train.tmp'
gcol_test_file = 'ALL_AML_gcol.test.tmp'
gcol_class_test_file = 'ALL_AML_gcol_class.test.csv'
gcol_class_train_file = 'ALL_AML_gcol_class.train.csv'
removable_word = 'endogenous control'

## Microarray Data Cleaning Steps

Remove Control from Gene Description and Replacing Commas with semicolon :

In [3]:
def remove_rows(file_path,tmp_path,removable):
    data = pd.read_csv(file_path, sep='\t',index_col=False)
    selected_rows = [g.find(removable) == -1 for g in data['Gene Description']]
    print('Number of occurrences of {} in {} : {}'.format(removable,file_path,np.sum(np.array(selected_rows) == False)))
    data = data[selected_rows]
    data.to_csv(tmp_path,index=False)
remove_rows(train_file,train_tmp_file,removable_word)
remove_rows(test_file,test_tmp_file,removable_word)

Number of occurrences of endogenous control in ALL_AML_grow.train.orig.txt : 58
Number of occurrences of endogenous control in ALL_AML_grow.test.orig.txt : 58


Remove unnecessary columns and renaming "Gene Accession Number" column to "ID" :

In [4]:
def remove_unnecessary_columns(file_path) :
    data = pd.read_csv(file_path)
    data = data[np.append(data.columns[1],data.columns[2::2])]
    data.columns = np.append(['ID'],data.columns[1:])
    return data
train_data = remove_unnecessary_columns(train_tmp_file)
test_data = remove_unnecessary_columns(test_tmp_file)

Limiting Attributes Value, between 20 and 16000 :

In [5]:
def normalize_data(data,save_path) :
    data = data.applymap(lambda x:x if type(x) == str or (x > 20 and x < 16000) else None)
    data.to_csv(save_path,index=False)
normalize_data(train_data,norm_train_file)
normalize_data(test_data,norm_test_file)

Transposing Matrix :

In [6]:
def tansposing_matrix(data_path,save_path) :
    data = pd.read_csv(data_path).transpose()
    print('Shape of {} : {}'.format(data_path,data.shape))
    data.to_csv(save_path)
tansposing_matrix(norm_train_file,gcol_train_file)
tansposing_matrix(norm_test_file,gcol_test_file)

Shape of ALL_AML_grow.train.norm.tmp : (39, 7071)
Shape of ALL_AML_grow.test.norm.tmp : (35, 7071)


Merging ClassId tables With Gcol tables :

In [30]:
pd.merge(pd.read_csv(gcol_test_file,header=1),
                   pd.read_csv(test_idclass_file),on="ID") \
                        .to_csv(gcol_class_test_file,index=False)
    
pd.merge(pd.read_csv(gcol_train_file,header=1),
                   pd.read_csv(train_idclass_file),on="ID") \
                        .to_csv(gcol_class_train_file,index=False)